In [50]:
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import gridspec
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
from pygadgetreader import readsnap, readhead, readheader
import re

In [59]:
# specify for which entries below to calculate the satellite positions at each redshift
#systems = [16, 17, 18, 19]
# systems = [0,1,2,3,4,5,6,7,8,9]
# systems = [0,1,2,3,4]
systems = [1]
# central galaxy index (i.e.) position in the Galform file (one entry per system)
# only needed for the systems where the central is not a type=0 galaxy with the largest halo mass
# index of the galaxy in the Galform file dataset 'Output001'

simulation_names = [
    "M1_zcut7", # 0
    "M2_zcut7",
    "M3_zcut7",
    "M4_zcut7",
    "M5_zcut7",
    "M1_7DM_GAS", # 5
    "M2_7DM_GAS",
    "M3_7DM_GAS",
    "M4_7DM_GAS",
    "M5_7DM_GAS"
]

centralIndeces = [
    0,    # M1_zcut7
    0,    # M2_zcut7
    0,    # M3_zcut7
    518,  # M4_zcut7
    79,   # M5_zcut7
    0,    # M1_7DM_GAS
    387,  # M2_7DM_GAS
    0,    # M3_7DM_GAS
    581,  # M4_7DM_GAS
    0,    # M5_7DM_GAS
]

# path to the input files
root_galformFiles = "/home/zhaox/Magpie/Galform/"
list_galformFiles = [
    "M1_zcut7/galaxies.hdf5",
    "M2_zcut7/galaxies.hdf5",
    "M3_zcut7/galaxies.hdf5",
    "M4_zcut7/galaxies.hdf5",
    "M5_zcut7/galaxies.hdf5",
    "M1_7DM_GAS/galaxies.hdf5",
    "M2_7DM_GAS/galaxies.hdf5",
    "M3_7DM_GAS/galaxies.hdf5",
    "M4_7DM_GAS/galaxies.hdf5",
    "M5_7DM_GAS/galaxies.hdf5",
]

# path to the input particle data files
root_subhaloFiles = "/home/zhaox/Magpie/"
list_subhaloFiles = [
    "M1_zcut7/groups_%03i_z???p???/",
    "M2_zcut7/groups_%03i_z???p???/",
    "M3_zcut7/groups_%03i_z???p???/",
    "M4_zcut7/groups_%03i_z???p???/",
    "M5_zcut7/groups_%03i_z???p???/",
    "M1_7DM_GAS/groups_%03i_z???p???/",
    "M2_7DM_GAS/groups_%03i_z???p???/",
    "M3_7DM_GAS/groups_%03i_z???p???/",
    "M4_7DM_GAS/groups_%03i_z???p???/",
    "M5_7DM_GAS/groups_%03i_z???p???/",
]

# path to the output 
root_satelliteFiles = "/home/zhaox/Magpie/Destruction/satellite_orbits_Galform/"
list_satelliteFiles = [
    "M1_MR.hdf5",
    "M2_MR.hdf5",
    "M3_MR.hdf5",
    "M4_MR.hdf5",
    "M5_MR.hdf5",
    "M1_7DM_GAS.hdf5",
    "M2_7DM_GAS.hdf5",
    "M3_7DM_GAS.hdf5",
    "M4_7DM_GAS.hdf5",
    "M5_7DM_GAS.hdf5",
]

# path to the output files for figures
plotFigures = True
root_plotFiles = "./fig/"
list_plotFiles = [
    "M1_MR_orbits.pdf",
    "M2_MR_orbits.pdf",
    "M3_MR_orbits.pdf",
    "M4_MR_orbits.pdf",
    "M5_MR_orbits.pdf",
    "M1_7DM_GAS_orbits.pdf",
    "M2_7DM_GAS_orbits.pdf",
    "M3_7DM_GAS_orbits.pdf",
    "M4_7DM_GAS_orbits.pdf",
    "M5_7DM_GAS_orbits.pdf",
]

if systems is None:
    systems = list(range(len(centralIndeces)))


dm_halo_minor_l = []

central_GroupNumber_l = [1,1,1,2,2,1,2,1,3,1]
r = 300 # kpc
# r = 2.174019391371011807e+02

In [60]:
def read_dm_pos(filedir, file_id, r, center_pos):
    for file in os.listdir(filedir):
        if re.search('snapshot_'+file_id, file) != None:
            filedir_snap = filedir + '/' + file
            file_list = np.array(os.listdir(filedir_snap))
            file_list.sort()
            snap_file = filedir_snap + '/' +file_list[0]
            print(snap_file)
            # a = readheader(snap_file, 'a')
            dm = readsnap(snap_file, 'pos', 'dm') * 1000/h
            dm = dm - center_pos
            dis = np.linalg.norm(dm, axis = 1) 
            dm = dm[dis<r]
    return dm

In [61]:
def read_satellite_orbit_Galform(satelliteFile):
# within 300kpc, > mass_min, after tidal destruction

    with h5py.File(satelliteFile, 'r' ) as hf:
        central_pos = hf['central_galaxy/pos'][:][0]
        h = hf['header_info/h'][()].astype(np.float64)
    return central_pos, h

In [62]:
def cal_c_a_ratio(satellite_coordinate):
    n = len(satellite_coordinate)
    I = np.zeros([3,3])
    for i in range(3):
        for j in range(3):
            for k in range(n):
                I[i][j] = I[i][j] + satellite_coordinate[k][i]*satellite_coordinate[k][j]
    # print(I)
    eigenvalue, featurevector = np.linalg.eig(I)
    # print("eigenvalue:", eigenvalue)
    # print("featurevector:", featurevector)
    c = min(eigenvalue)**0.5
    a = max(eigenvalue)**0.5
    rng=range(len(eigenvalue))
    mindex=min(rng,key=lambda x:eigenvalue[x])
    c_a_minor = featurevector[:,mindex]
    # print("c/a=", c/a)
    return c/a, c_a_minor

In [63]:
file_id = '199_z000p000'
for system in systems:
    print(system)
    satelliteFile = root_satelliteFiles + list_satelliteFiles[system]
    central_pos, h = read_satellite_orbit_Galform(satelliteFile)
    simulation_name = simulation_names[system]
    filedir = '/data/inspur_disk02/Simulations/Magpie/' + simulation_name
    dm_pos = read_dm_pos(filedir, file_id, r, central_pos)
    c_a, c_a_minor = cal_c_a_ratio(dm_pos)
    dm_halo_minor_l.append(c_a_minor)

1
/data/inspur_disk02/Simulations/Magpie/M2_zcut7/snapshot_199_z000p000/snap_199_z000p000.0.hdf5
Returning DM    Positions


In [64]:
print(c_a_minor)

[-0.18031118 -0.48783585  0.8541101 ]


In [65]:
dm_halo_minor_l

[array([-0.18031118, -0.48783585,  0.8541101 ])]